<a href="https://colab.research.google.com/github/bbussiere-botify/tmpVCO/blob/main/Hackaton_2024_PW_Copilot_Generate_titles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# @title Imports
from google.colab import auth
from google.cloud import bigquery

PROJECT = 'botify-bigquery-flat'
LOCATION = 'US'

client = bigquery.Client(project=PROJECT, location=LOCATION)

auth.authenticate_user()

In [ ]:
project_id = 56398
input_urls_table_name = f"botify-analytics-staging.temp.input_{project_id}"

In [ ]:
# metadata_query = f"""
#     SELECT
#     project_id,
#     slug,
#     analysis_slug,
#     activation_website_id,
#     internal_links_table,
#     crawl_table,
#     gsc_table,
#     -- IFNULL(behaviors_airbyte_table,behaviors_table) behaviors_table
#     FROM `botify-analytics-prod.dbt_marts_botify_metadata.analysis` a
#     LEFT JOIN `botify-analytics-prod.dbt_marts_botify_metadata.projects_active` USING (project_id)
#     WHERE 1=1
#     AND DATE(analysis_date_created) >= "2024-05-01"
#     AND internal_links_table IS NOT NULL
#     AND project_type = "webproperty_main"
#     AND analysis_type = 1
#     AND analysis_crawler_status = 5
#     AND analysis_workflow_status = 10
#     AND slug LIKE "%{project_slug}%"
#     -- AND DATE(logs_max_date) >= "2024-04-01"
#     QUALIFY
#     ROW_NUMBER() OVER (PARTITION BY project_id ORDER BY a.urls_done DESC) = 1
#     """

In [ ]:
input_urls_query = f"""
    CREATE OR REPLACE TABLE `{input_urls_table_name}` AS
    SELECT
        url_hash,
    FROM (
        SELECT
        ANY_VALUE(COALESCE(sq_3__url, sq_2__url, sq_1__url)) url,
        ANY_VALUE(COALESCE(sq_3__url_hash, sq_2__url_hash, sq_1__url_hash)) url_hash,
        sq_2__crawl__20240528__metadata__description__content f_1,
        IFNULL(SUM(sq_3__search_console__period_0__count_impressions), 0) f_2,
        IFNULL(SUM(sq_3__search_console__period_0__count_clicks), 0) f_3,
        ((SUM(sq_3__search_console__period_0__count_clicks) / SUM(sq_3__search_console__period_0__count_impressions)) * 100) f_4,
        (SUM((sq_3__search_console__period_0__count_impressions * sq_3__search_console__period_0__avg_position)) / SUM(sq_3__search_console__period_0__count_impressions)) f_5
        FROM (
        SELECT
            sq_1.url sq_1__url,
            sq_1.url_hash sq_1__url_hash,
            sq_2.crawl__20240528__metadata__description__content sq_2__crawl__20240528__metadata__description__content,
            sq_2.url sq_2__url,
            sq_2.url_hash sq_2__url_hash,
            sq_3.search_console__period_0__avg_position sq_3__search_console__period_0__avg_position,
            sq_3.search_console__period_0__count_clicks sq_3__search_console__period_0__count_clicks,
            sq_3.search_console__period_0__count_impressions sq_3__search_console__period_0__count_impressions,
            sq_3.url sq_3__url,
            sq_3.url_hash sq_3__url_hash
        FROM (
            SELECT
            ANY_VALUE(path) path,
            ANY_VALUE(url) url,
            date,
            url_hash
            FROM (
            SELECT
                *
            FROM
                `botify-production-15.production_56398.trended_crawls`
            WHERE
                PARTITIONTIME BETWEEN DATE_TRUNC('2024-05-07', DAY)
                AND DATE_TRUNC('2024-05-28', DAY)
                AND http_code <> 0)
            WHERE
            path <> '/'
            GROUP BY
            date,
            url_hash) sq_1
        FULL OUTER JOIN (
            SELECT
            (SUM((impressions * avg_position)) / SUM(impressions)) search_console__period_0__avg_position,
            ANY_VALUE(branded) branded,
            ANY_VALUE(path) path,
            ANY_VALUE(url) url,
            ANY_VALUE(url_hash) url_hash,
            search_type,
            SUM(clicks) search_console__period_0__count_clicks,
            SUM(impressions) search_console__period_0__count_impressions,
            url_keyword_hash
            FROM (
            SELECT
                * EXCEPT (search_type),
                IFNULL(search_type, 'WEB') search_type
            FROM
                `botify-production-15.production_56398.search_console_flat`
            WHERE
                date BETWEEN '2024-05-07'
                AND '2024-06-03')
            WHERE
            path <> '/'
            GROUP BY
            search_type,
            url_keyword_hash) sq_3
        ON
            sq_1.url_hash = sq_3.url_hash
        FULL OUTER JOIN (
            SELECT
            *,
            metadata__description__contents[SAFE_OFFSET(0)] crawl__20240528__metadata__description__content
            FROM
            `botify-production-15.production_56398.crawl_1526437_1604378`
            WHERE
            http_code <> 0
            AND path <> '/') sq_2
        ON
            COALESCE(sq_1.url_hash, sq_3.url_hash) = sq_2.url_hash
        WHERE
            sq_3.search_type = 'WEB'
            AND sq_2.depth < 4
            AND sq_2.internal_page_rank__value > 6
            AND sq_2.lang = 'fr-FR'
            AND sq_2.compliant__is_compliant
            AND NOT(sq_3.branded)
            AND sq_1.url_hash IS NOT NULL)
        GROUP BY
        COALESCE(sq_3__url_hash, sq_2__url_hash, sq_1__url_hash),
        f_1
        ORDER BY
        4 DESC
    )
    """

In [ ]:
def generate_urls_metrics_query(
        input_urls_table_name,
        min_impressions=1000,
        nb_limit_keywords=3,
        gsc_table_name = "botify-production-15.production_56398.search_console_flat",
        trended_crawls_table_name = "botify-production-15.production_56398.trended_crawls"
        ):
    """Generate a query to get"""
    return f"""
    WITH

    input_urls AS (SELECT url_hash FROM `{input_urls_table_name}`),

    gsc AS (
    SELECT
        url,
        keyword,
        ANY_VALUE(url_hash) url_hash,
        SUM(clicks) clicks,
        SUM(impressions) impressions,
        SAFE_DIVIDE(SUM(clicks), SUM(impressions)) ctr
    FROM `{gsc_table_name}`
    WHERE
    date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY) AND CURRENT_DATE()
    AND branded IS FALSE -- Only unbranded excluding anon.
    GROUP BY url, keyword
    HAVING impressions >= {min_impressions}
    ),

    gsc_top_three_keywords_source AS (
    SELECT
    *
    FROM gsc
    QUALIFY
        ROW_NUMBER() OVER (PARTITION BY url ORDER BY ctr DESC) <= {nb_limit_keywords}
    ),

    gsc_top_three_keywords AS (
    SELECT
        url,
        ANY_VALUE(url_hash) url_hash,
        ARRAY_AGG(keyword) top_three_keywords,
        SUM(clicks) clicks,
        SUM(impressions) impressions,
        SAFE_DIVIDE(SUM(clicks), SUM(impressions)) ctr
    FROM gsc_top_three_keywords_source
    GROUP BY url
    ORDER BY impressions DESC
    ),

    crawls AS (
    SELECT
        url_hash,
        ANY_VALUE(metadata__title__contents)[SAFE_OFFSET(0)] metadata__title__contents,
        ANY_VALUE(metadata__description__contents)[SAFE_OFFSET(0)] metadata__description__contents,
        ANY_VALUE(metadata__h1__contents)[SAFE_OFFSET(0)] metadata__h1__contents,
    FROM `{trended_crawls_table_name}`
    WHERE DATE(PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY) AND CURRENT_DATE()
    GROUP BY url_hash
    )

    SELECT
    url,
    top_three_keywords keywords,
    metadata__title__contents title,
    metadata__description__contents description,
    metadata__h1__contents h1,
    FROM gsc_top_three_keywords AS gsc
    INNER JOIN crawls USING(url_hash)
    INNER JOIN input_urls USING(url_hash)
    """

In [ ]:
urls_metrics_query = generate_urls_metrics_query(input_urls_table_name)

In [ ]:
# Create input urls
#client.query(input_urls_query).result()

In [ ]:
df = client.query(urls_metrics_query).to_dataframe()

In [ ]:
query = """
WITH

gsc AS (
SELECT
    url,
    keyword,
    SUM(impressions) impressions,
    LENGTH(keyword) keyword_length,
FROM `botify-production-15.production_56398.search_console_flat`
WHERE 1=1
AND url like '%https://fr.%'
AND date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY) AND CURRENT_DATE()
AND branded IS FALSE -- Only unbranded excluding anon.
GROUP BY url, keyword
HAVING impressions >= 500
ORDER BY keyword_length DESC
)


SELECT
url,
STRING_AGG(keyword LIMIT 5) keywords,
SUM(impressions) impressions,
FROM gsc
GROUP BY url
ORDER BY impressions DESC
LIMIT 1000
"""

In [ ]:
links_df = client.query(query).to_dataframe()

In [ ]:
links_df.shape

(1000, 3)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import make_pipeline
from scipy import sparse
import numpy as np
from sklearn.metrics import pairwise_distances_chunked
from itertools import chain
from vertexai.preview.language_models import TextEmbeddingModel
import vertexai

vertexai.init(project="botify-hackaton-2024-2", location="us-central1")
model = TextEmbeddingModel.from_pretrained("text-multilingual-embedding-002")

def _create_ngrams(string):
   result = []
   for n in range(3, 4):
      ngrams = zip(*[string[i:] for i in range(n)])
      ngrams = [''.join(ngram) for ngram in ngrams if ' ' not in ngram]

   result.extend(ngrams)
   return(result)


def get_embeddings(x):
    vectorizer = TfidfVectorizer(analyzer=_create_ngrams)
    return vectorizer.fit_transform(x)

def get_embeddings(x):
    from itertools import chain

    x_chunks = [x[i:i + 200] for i in range(0, len(x), 200)]
    embeddings_chunks= []
    for chunk in x_chunks:
        embeddings_chunk = model.get_embeddings(list(chunk))
        embeddings_chunks.append(embeddings_chunk)

    embeddings = list(chain(*embeddings_chunks))
    embeddings = np.array([e.values for e in embeddings])
    return embeddings


def get_cosine_chunks(embeddings,threshold, metric):
    # We are calculating distances and not similarities
    threshold =  1 - threshold
    for chunk in pairwise_distances_chunked(embeddings, metric=metric):
        # Get similar vectors given a threshold
        similar_vectors = np.array(chunk<=threshold)
        # Yield only the indexes that respects the threshold
        yield (np.where(r)[0] for r in similar_vectors)


def get_similar_indexes(embeddings,threshold, metric):
    """
    Get the cosine chunks. As chunks are ordered, we will chain every
     array contaning the most similar indexes.
     The results: for every embedding we have their most similar embeddings indexes
    """
    return list(chain((rr for r in get_cosine_chunks(embeddings,threshold, metric) for rr in r)))


def get_similar_indexes_mapping(similar_indexes):
    """Want to keep a query only once """
    n = len(similar_indexes)
    seen_index = set()
    res = {}
    for i in range(n):
        if i not in seen_index:
            similar_index = similar_indexes[i]
            similar_index_updated = set(similar_index) - seen_index
            seen_index.update(similar_index)
            res[i] = similar_index_updated
    return {vv: int(k) for k,v in res.items() for vv in v}


def get_similar_text_mapping(input_txt, similar_indexes):
    similar_indexes_mapping = get_similar_indexes_mapping(similar_indexes)
    return {input_txt[k]: input_txt[v] for k,v in similar_indexes_mapping.items()}


def group_similar_sentences(df, input_col_name, output_col_name, threshold ,metric= 'cosine'):
    input_txt = df[input_col_name].unique()
    embeddings = get_embeddings(input_txt)
    similar_indexes = get_similar_indexes(embeddings,threshold, metric)
    mapping = get_similar_text_mapping(input_txt, similar_indexes)
    df[output_col_name] = df[input_col_name].map(mapping).fillna('-1')
    return df

In [ ]:
links_df = links_df.pipe(group_similar_sentences, 'keywords', 'group', .90, metric="cosine")
groups_df = links_df.groupby('group').agg({"keywords":set, "url":set, "impressions":"sum"}).sort_values(by="impressions", ascending=False).reset_index()
groups_df['len_urls']  = groups_df['url'].apply(len)
groups_df.sort_values('len_urls', ascending=False)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import make_pipeline
from scipy import sparse
import numpy as np
from sklearn.metrics import pairwise_distances_chunked
from itertools import chain
from vertexai.preview.language_models import TextEmbeddingModel
import vertexai

vertexai.init(project="botify-hackaton-2024-2", location="us-central1")
model = TextEmbeddingModel.from_pretrained("text-multilingual-embedding-002")

def _create_ngrams(string):
   result = []
   for n in range(3, 4):
      ngrams = zip(*[string[i:] for i in range(n)])
      ngrams = [''.join(ngram) for ngram in ngrams if ' ' not in ngram]

   result.extend(ngrams)
   return(result)


def get_embeddings(x):
    vectorizer = TfidfVectorizer(analyzer=_create_ngrams)
    return vectorizer.fit_transform(x)


def get_cosine_chunks(embeddings,threshold, metric):
    # We are calculating distances and not similarities
    threshold =  1 - threshold
    for chunk in pairwise_distances_chunked(embeddings, metric=metric):
        # Get similar vectors given a threshold
        similar_vectors = np.array(chunk<=threshold)
        # Yield only the indexes that respects the threshold
        yield (np.where(r)[0] for r in similar_vectors)


def get_similar_indexes(embeddings,threshold, metric):
    """
    Get the cosine chunks. As chunks are ordered, we will chain every
     array contaning the most similar indexes.
     The results: for every embedding we have their most similar embeddings indexes
    """
    return list(chain((rr for r in get_cosine_chunks(embeddings,threshold, metric) for rr in r)))


def get_similar_indexes_mapping(similar_indexes):
    """Want to keep a query only once """
    n = len(similar_indexes)
    seen_index = set()
    res = {}
    for i in range(n):
        if i not in seen_index:
            similar_index = similar_indexes[i]
            similar_index_updated = set(similar_index) - seen_index
            seen_index.update(similar_index)
            res[i] = similar_index_updated
    return {vv: int(k) for k,v in res.items() for vv in v}


def get_similar_text_mapping(input_txt, similar_indexes):
    similar_indexes_mapping = get_similar_indexes_mapping(similar_indexes)
    return {input_txt[k]: input_txt[v] for k,v in similar_indexes_mapping.items()}


def group_similar_sentences(df, input_col_name, output_col_name, threshold ,metric= 'cosine'):
    input_txt = df[input_col_name].unique()
    embeddings = get_embeddings(input_txt)
    similar_indexes = get_similar_indexes(embeddings,threshold, metric)
    mapping = get_similar_text_mapping(input_txt, similar_indexes)
    df[output_col_name] = df[input_col_name].map(mapping).fillna('-1')
    return df

In [ ]:
links_df1 = links_df.pipe(group_similar_sentences, 'keywords', 'group', .65, metric="cosine").copy()
groups_df1 = links_df1.groupby('group').agg({"keywords":set, "url":set, "impressions":"sum"}).sort_values(by="impressions", ascending=False).reset_index()
groups_df1['len_urls']  = groups_df1['url'].apply(len)

In [ ]:
groups_df1  = groups_df1[groups_df1['len_urls'] > 1].sort_values('len_urls', ascending=False).reset_index(drop=True)

In [ ]:
groups_df1

,group,keywords,url,impressions,len_urls
0,vintage,"{adidas vintage homme,adidas vintage, vintage,...",{https://fr.vestiairecollective.com/vintage/bo...,50278.0,13
1,"pochette dior femme,dior pochette,pochette dior","{pochette dior femme,dior pochette,pochette di...",{https://fr.vestiairecollective.com/sacs-femme...,58142.0,12
2,"sac sunny zadig et voltaire,sac zadig et volta...","{pull zadig et voltaire, sac zadig et voltaire...",{https://fr.vestiairecollective.com/vetements-...,25459.0,12
3,"balenciaga track chaussures,balenciaga track s...","{pull balenciaga, balenciaga track chaussures,...",{https://fr.vestiairecollective.com/chaussures...,88021.0,12
4,"sac jacquemus pas cher,sac a main jacquemus,sa...","{tee shirt jacquemus homme,t shirt jacquemus h...",{https://fr.vestiairecollective.com/sacs-femme...,97460.0,12
...,...,...,...,...,...
171,cos,"{cos homme, cos}","{https://fr.vestiairecollective.com/cos/, http...",9264.0,2
172,"balenciaga triple s sneakers,balenciaga triple...","{balenciaga triple s sneakers,balenciaga tripl...",{https://fr.vestiairecollective.com/chaussures...,9304.0,2
173,"casquette celine femme,casquette celine,celine...","{casquette celine femme,casquette celine,celin...",{https://fr.vestiairecollective.com/accessoire...,10056.0,2
174,"bracelet homme fred,bracelet fred homme,fred b...","{bracelet homme fred,bracelet fred homme,fred ...",{https://fr.vestiairecollective.com/accessoire...,10549.0,2


In [ ]:
list(groups_df1.iloc[1])

['pochette dior femme,dior pochette,pochette dior',
 {'louis vuitton pochette,pochette louis vuitton,pochette vuitton,pochette lv',
  'pochette chanel',
  'pochette dior en paille,pochette dior paille',
  'pochette dior femme,dior pochette,pochette dior',
  'pochette dior paille',
  'pochette gucci femme,pochettes gucci,pochette gucci',
  'pochette hermes',
  'pochette yves saint laurent,pochette saint laurent,pochette ysl femme,pochette ysl'},
 {'https://fr.vestiairecollective.com/sacs-femme/dior/paille/',
  'https://fr.vestiairecollective.com/sacs-femme/louis-vuitton/pochette-accessoire/',
  'https://fr.vestiairecollective.com/sacs-femme/pochettes/chanel/',
  'https://fr.vestiairecollective.com/sacs-femme/pochettes/chanel/pochette-chanel-wallet-on-chain-en-cuir-noir-39705322.shtml',
  'https://fr.vestiairecollective.com/sacs-femme/pochettes/christian-dior/',
  'https://fr.vestiairecollective.com/sacs-femme/pochettes/dior/paille/',
  'https://fr.vestiairecollective.com/sacs-femme/poch